# A1111 Model Installer — SD 1.5 (UI)

Installs **SD 1.5** assets for Automatic1111 using a simple UI:

- **Checkpoints** (paste URLs) → `/workspace/a1111/models/Stable-diffusion/`
- **VAE (optional)** → `/workspace/a1111/models/VAE/`
- **ControlNet (SD15)** → `/workspace/a1111/models/ControlNet/`

Re-running is safe: existing files are skipped; existing repos are updated.

> If a model is gated on Hugging Face, set `HF_TOKEN` in the RunPod template.

In [ ]:
from pathlib import Path
import os

# A1111 data root (matches WEBUI_ROOT in your image)
DATA_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"

for d in (CKPT_DIR, VAE_DIR, CN_DIR):
    d.mkdir(parents=True, exist_ok=True)
print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet (SD15):", CN_DIR)

## Helpers (with progress bars)

In [ ]:
import requests, shutil, os
from tqdm import tqdm
from huggingface_hub import hf_hub_download
from pathlib import Path

def human(n: int) -> str:
    for unit in ["B","KB","MB","GB"]:
        if n < 1024: return f"{n:.1f} {unit}"
        n /= 1024
    return f"{n:.1f} TB"

def download_direct(url: str, dest: Path):
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        print("✔ Exists:", dest); return dest
    r = requests.get(url, stream=True)
    r.raise_for_status()
    total = int(r.headers.get('Content-Length', 0))
    tmp = dest.with_suffix(dest.suffix + ".part")
    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk); bar.update(len(chunk))
    tmp.replace(dest)
    print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")
    return dest

def download_hf(repo_id, filename, dest_dir, dest_name=None):
    dest = dest_dir / (dest_name or filename)
    if dest.exists(): print("✔ Exists:", dest); return dest
    print(f"→ HF: {repo_id}/{filename}")
    path = hf_hub_download(repo_id=repo_id, filename=filename, token=os.getenv('HF_TOKEN'), local_dir=dest_dir)
    shutil.copy2(path, dest)
    print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")

## Pick what to install
Paste SD 1.5 **checkpoint URLs**, toggle VAE or ControlNets, then click **Download Selected**.

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

vae_checkbox = W.Checkbox(description="Install SD1.5 MSE VAE (stabilityai/sd-vae-ft-mse)", value=False)
cn_options = [
    ("Canny",     "lllyasviel/control_v11p_sd15_canny"),
    ("SoftEdge",  "lllyasviel/control_v11p_sd15_softedge"),
    ("OpenPose",  "lllyasviel/control_v11p_sd15_openpose")
]
cn_checkboxes = [W.Checkbox(description=label, value=True) for label, _ in cn_options]
ckpt_text = W.Textarea(placeholder="Paste .safetensors URLs, one per line", layout=W.Layout(width="100%", height="120px"))

btn = W.Button(description="Download Selected", button_style="success", icon="download")
out = W.Output()

def on_click(_):
    out.clear_output()
    with out:
        errors = []
        if vae_checkbox.value:
            download_hf("stabilityai/sd-vae-ft-mse", "vae-ft-mse-840000-ema-pruned.safetensors", VAE_DIR)
        for cb, (label, repo) in zip(cn_checkboxes, cn_options):
            if cb.value:
                download_hf(repo, "diffusion_pytorch_model.safetensors", CN_DIR, f"control_{label.lower()}.safetensors")
        for url in [u.strip() for u in ckpt_text.value.splitlines() if u.strip()]:
            download_direct(url, CKPT_DIR / url.split('/')[-1])
        print("✔ Done. Refresh WebUI model list if open.")

btn.on_click(on_click)
display(vae_checkbox)
display(Markdown("### ControlNet (SD15)"))
for c in cn_checkboxes: display(c)
display(Markdown("### Checkpoints"))
display(ckpt_text, btn, out)